In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일 자동으로 루트에서 찾아서 로드
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings

# PDF 로딩
loader = PyPDFLoader("../data/sleep_performance.pdf")
documents = loader.load_and_split()

# 임베딩 생성
embedding = OpenAIEmbeddings()  # .env에 API 키 필요!
vectorstore = FAISS.from_documents(documents, embedding)

# 저장
vectorstore.save_local("vectorstores/sleep_rag")


In [4]:
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 벡터 DB 로딩
vectorstore = FAISS.load_local(
    "vectorstores/sleep_rag",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# LLM 설정
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# RAG 체인 생성
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 질의 실행
query = "문서의 내용을 요약해줘"
response = rag_chain(query)
print(response["result"])


C:\Users\user\AppData\Local\Temp\ipykernel_19836\1859890952.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
C:\Users\user\AppData\Local\Temp\ipykernel_19836\1859890952.py:25: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain(query)


이 연구는 수면부족이 운동 성능에 미치는 영향을 조사한 것으로, 670명의 참가자를 대상으로 45개의 연구를 분석하였습니다. 연구에서는 유산소 및 무산소 지구력, 최대 힘, 폭발적 힘, 속도, 기술 통제, 지각된 노력 등의 결과 지표를 분석하였습니다. 수면부족은 유산소 지구력에 부정적인 영향을 미치는 것으로 나타났으며, 특히 오후 시간대에 테스트를 실시한 경우 더 큰 영향을 미쳤습니다. 무산소 지구력에 대한 영향은 통계적으로 유의하지 않았습니다. 연구에서는 다양한 요인들이 수면부족이 운동 성능에 미치는 영향을 조사하기 위해 부속 분석을 실시하였습니다.
